In [1]:
import pyautogui as ag
import PySimpleGUI as sg
import time
import pandas as pd
from win32gui import GetWindowText, GetForegroundWindow

sg.theme('DarkGreen')
ag.PAUSE = 0.001

In [5]:
def paste_txt(text, x, y):
    ag.doubleClick(x,y)
    ag.press('del')
    ag.write(str(text).replace(' ', '_'), interval=0.0001)

def open_extended():
    ag.click(150,420)
    time.sleep(0.2)
    ag.click(150,450)
    

In [3]:
def paste_MetaData(SampleName, SpeciesName, OtherInfo, CollectionDate, Station, Location, Lat, Long, Determiner, Sex):
    paste_txt(SampleName, 170,130)
    paste_txt(SpeciesName, 30,230)
    paste_txt(OtherInfo, 30,280)
    open_extended()
    paste_txt(CollectionDate, 30,470)
    paste_txt(Station, 30,520)
    paste_txt(Location, 30,570)
    paste_txt(Lat, 30,620)
    paste_txt(Long, 210,620)
    paste_txt(Determiner, 30,670)
    paste_txt(Sex, 30,720)


In [7]:
layout = [  [sg.T('Automatically insert metadata to SRS')],
            [sg.T('Choose input file: '), sg.FileBrowse(key='-FILE-', file_types=(('xlsx', '*.xlsx'),)), sg.B('Read')],
            [sg.T(key='-LoadedFILE-'), sg.T(key='-FILEinfo-')],
            [sg.T('Start from excel sheet row'), sg.Input(key='-starting_line-'), sg.B('Ok')],
            [sg.T(' ')],
            [sg.T(size=(45,2), key='-WARNING-'), sg.B('Next Line')],
            [sg.B('Quit')]]

# Create the window
window = sg.Window('MetaDataManagingApplication', layout, keep_on_top=True)
count = 0

# Display and interact with the Window using an Event Loop
while True:
    event, values = window.read()
    # See if user wants to quit or window was closed
    if event == sg.WINDOW_CLOSED or event == 'Quit':
        break
    # When pressing 'Read' button: Check if file has been selected. If yes, load excel file.
    if event == 'Read':
        if values['-FILE-'] == '':
            window['-LoadedFILE-'].update('Please select a file')
        else:
            loaded_data = pd.read_excel(values['-FILE-'], keep_default_na=False)
            window['-LoadedFILE-'].update('Loaded ' + str(*values['-FILE-'].split('/')[-1:]))
            window['-FILEinfo-'].update('containing ' + str(len(loaded_data.index)) + ' rows (without headline)')

    # When pressing 'Ok' button: Check if input was int. If yes, update starting row.
    if event == 'Ok':
        if values['-starting_line-'] == '':
            window['-WARNING-'].update('Missing Input')
        else:
            try: 
                int(values['-starting_line-']) == int
                window['-WARNING-'].update('Starting from row: ' + str(*values['-starting_line-']))
                count = int(values['-starting_line-']) - 2
                if count <= -1:
                    window['-WARNING-'].update('Invalid line input: Headline cannot be pasted')
                if count + 1 > len(loaded_data.index):
                    window['-WARNING-'].update('Invalid line input: File only has ' + str(len(loaded_data.index)+1) + ' rows')
                else:
                    pass
            except: 
                window['-WARNING-'].update('Invalid line input')

    # When pressing 'Next Line' button: Check if file has been selected. If yes, check if SRS window can be selected. If yes, paste meta data.
    if event == 'Next Line':
        if values['-FILE-'] == '':
            window['-WARNING-'].update('No file selected')
        try: 
            loaded_data
            if count >= len(loaded_data.index):
                window['-WARNING-'].update('End of excel file reached')
            else:

                    ag.click(1000,15)
                    if GetWindowText(GetForegroundWindow()) == 'SRS':
                        window['-WARNING-'].update('pasting...')
                        paste_MetaData(*loaded_data.iloc[count])
                        count +=1
                        window['-WARNING-'].update('Info succesfully pasted')
                    else:
                        window['-WARNING-'].update('SRS window could not be selected')
        except:
            window['-WARNING-'].update('Data not loaded. Read data first')

# Finish up by removing from the screen
window.close()